In [1]:
from datetime import datetime

from models import FineTuner
from generate_metrics import Metric

/home/bhatt/miniconda3/envs/thesis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
(datetime.now()).strftime("%Y-%m-%d")

'2023-12-19'

In [3]:
%load_ext autoreload
%autoreload 2

In [27]:
model_checkpoint = "bert-base-cased"
model_save_dir = f"/home/bhatt/ishan/TUM_Thesis/data/models/{model_checkpoint}_"+(datetime.now()).strftime("%Y-%m-%d")
ft_model_save_dir = f"/home/bhatt/ishan/TUM_Thesis/data/models/{model_checkpoint}-finetuned_"+(datetime.now()).strftime("%Y-%m-%d")

In [5]:
ft_model_save_dir
dataset_name = 'imdb'
ft_model_save_dir = f"/home/bhatt/ishan/TUM_Thesis/data/models/{model_checkpoint}-finetuned-{dataset_name}_"+(datetime.now()).strftime("%Y-%m-%d")
ft_model_save_dir

'/home/bhatt/ishan/TUM_Thesis/data/models/bert-base-cased-finetuned-imdb_2023-12-19'

In [11]:
ft = FineTuner(model_name=model_checkpoint,from_local=True, model_save_dir=model_save_dir,local_model_path='/home/bhatt/ishan/TUM_Thesis/data/models/bert-base-cased-finetuned-imdb_2023-12-07/checkpoint-1500')

Initializing Model


Model Initialized!


In [12]:
ft.model_dir, ft.model_tag

('/home/bhatt/ishan/TUM_Thesis/data/models/bert-base-cased-finetuned-imdb_2023-12-07/checkpoint-1500',
 'bert-base-cased')

In [13]:
# ft.finetune_model(dataset_name = 'imdb')

In [23]:
metric = Metric(model_name = 'bert-base-cased', model = ft.getModel(), tokenizer = ft.getTokenizer(), model_tag= ft.model_tag, model_dir = ft.model_dir)

In [26]:
metric.get_metric(metric = 'crows-pairs', input_file = '/home/bhatt/ishan/TUM_Thesis/data/metrics_ds/crows-pairs/data/crows_pairs_anonymized.csv', output_file = '/home/bhatt/ishan/TUM_Thesis/data/results/cp_results.csv')

  0%|          | 1/1508 [00:00<09:50,  2.55it/s]/home/bhatt/ishan/TUM_Thesis/src/metrics/crows_pairs.py:266: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_score = pd.concat([df_score,pd.DataFrame({'sent_more': [sent_more],
100%|██████████| 1508/1508 [03:54<00:00,  6.44it/s]


Total examples: 1508
Metric score: 54.58
Stereotype score: 55.97
Anti-stereotype score: 46.33
Num. neutral: 0 0.0

Metrics Saved in  /home/bhatt/ishan/TUM_Thesis/data/results/master_results.xlsx


In [25]:
metric.get_metric(metric = 'stereoset', input_file = '/home/bhatt/ishan/TUM_Thesis/data/metrics_ds/stereoset/dev.json', output_file = 'stereoset_results.txt',
                   output_dir = '/home/bhatt/ishan/TUM_Thesis/data/results')

100%|██████████| 8939/8939 [00:52<00:00, 168.79it/s]


Metrics Saved in  /home/bhatt/ishan/TUM_Thesis/data/results/master_results.xlsx
